In [1]:
import pandas as pd
import os
import re
from tqdm import notebook, trange, tqdm
import ast
import numpy as np
from fuzzywuzzy import fuzz
import itertools
import logging
from seqeval.metrics import f1_score, precision_score, recall_score
import argparse
import glob
import random
import sys
import io
from transformers import (
    MODEL_FOR_TOKEN_CLASSIFICATION_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
    BertTokenizer, 
    BertForTokenClassification, 
    BertModel, 
    AutoModelWithLMHead,
)
import torch
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertAdam
from nltk.corpus import conll2002
try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter


D:\Anaconda\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:544: FutureWarning: Passing (type, 1) or '1type' 

In [2]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#n_gpu = torch.cuda.device_count()
#torch.cuda.get_device_name(0) 

## BERTje

In [3]:
def preprocess(data, max_len, prev, counter, hold_list, tokenizer):
    holder = hold_list
    if len(data[prev:]) == 0:
        return holder
    
    data_text = data[prev:]
    subword_len = []
    line = data_text.rstrip()
    line = line.split(' ')
    if len(line) == 1:
        text = '\n'.join(line)
        holder.append(text)
        return holder
    subword_len_counter = 0
    
    for x in line:
        current_subwords_len = len(''.join(tokenizer.tokenize(x)))
        if (subword_len_counter + current_subwords_len) > max_len:
            text = '\n'.join(subword_len)
            holder.append(text)
            return preprocess(data, max_len, prev + len(text), counter + 1, holder, tokenizer)
        else:
            subword_len.append(x)
            subword_len_counter += current_subwords_len
    
    text = '\n'.join(subword_len)    
    holder.append(text)

        
    return preprocess(data, max_len, prev + len(text), counter + 1, holder, tokenizer)
    

In [4]:
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Named entity recognition fine-tuning: utilities to work with CoNLL-2003 task. """


logger = logging.getLogger(__name__)


class InputExample(object):
    """A single training/test example for token classification."""

    def __init__(self, guid, words, labels):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            words: list. The words of the sequence.
            labels: (Optional) list. The labels for each word of the sequence. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.words = words
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids


def read_examples_from_file(data_dir, mode):
    guid_index = 1
    examples = []
    
    words = []
    labels = []
    for line in mode.split('\n'):
        if line.startswith("-DOCSTART-") or line == "" or line == "\n":
            if words:
                examples.append(InputExample(guid="{}-{}".format(mode, guid_index), words=words, labels=labels))
                guid_index += 1
                words = []
                labels = []
        else:
            splits = line.split(" ")
            words.append(splits[0])
            if len(splits) > 1:
                labels.append(splits[-1].replace("\n", ""))
            else:
                # Examples could have no label for mode = "test"
                labels.append("O")
    if words:
        examples.append(InputExample(guid="{}-{}".format(mode, guid_index), words=words, labels=labels))

    return examples


def convert_examples_to_features(
    examples,
    label_list,
    max_seq_length,
    tokenizer,
    cls_token_at_end=False,
    cls_token="[CLS]",
    cls_token_segment_id=1,
    sep_token="[SEP]",
    sep_token_extra=False,
    pad_on_left=False,
    pad_token=0,
    pad_token_segment_id=0,
    pad_token_label_id=-100,
    sequence_a_segment_id=0,
    mask_padding_with_zero=True,
):
    """ Loads a data file into a list of `InputBatch`s
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """

    label_map = {label: i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            logger.info("Writing example %d of %d", ex_index, len(examples))

        tokens = []
        label_ids = []
        for word, label in zip(example.words, example.labels):
            word_tokens = tokenizer.tokenize(word)

            # bert-base-multilingual-cased sometimes output "nothing ([]) when calling tokenize with just a space.
            if len(word_tokens) > 0:
                tokens.extend(word_tokens)
                # Use the real label id for the first token of the word, and padding ids for the remaining tokens
                label_ids.extend([label_map[label]] + [pad_token_label_id] * (len(word_tokens) - 1))

        # Account for [CLS] and [SEP] with "- 2" and with "- 3" for RoBERTa.
        special_tokens_count = tokenizer.num_added_tokens()
        if len(tokens) > max_seq_length - special_tokens_count:
            tokens = tokens[: (max_seq_length - special_tokens_count)]
            label_ids = label_ids[: (max_seq_length - special_tokens_count)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids:   0   0   0   0  0     0   0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens += [sep_token]
        label_ids += [pad_token_label_id]
        if sep_token_extra:
            # roberta uses an extra separator b/w pairs of sentences
            tokens += [sep_token]
            label_ids += [pad_token_label_id]
        segment_ids = [sequence_a_segment_id] * len(tokens)

        if cls_token_at_end:
            tokens += [cls_token]
            label_ids += [pad_token_label_id]
            segment_ids += [cls_token_segment_id]
        else:
            tokens = [cls_token] + tokens
            label_ids = [pad_token_label_id] + label_ids
            segment_ids = [cls_token_segment_id] + segment_ids

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_length - len(input_ids)
        if pad_on_left:
            input_ids = ([pad_token] * padding_length) + input_ids
            input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
            segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
            label_ids = ([pad_token_label_id] * padding_length) + label_ids
        else:
            input_ids += [pad_token] * padding_length
            input_mask += [0 if mask_padding_with_zero else 1] * padding_length
            segment_ids += [pad_token_segment_id] * padding_length
            label_ids += [pad_token_label_id] * padding_length

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        assert len(label_ids) == max_seq_length

        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s", example.guid)
            logger.info("tokens: %s", " ".join([str(x) for x in tokens]))
            logger.info("input_ids: %s", " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s", " ".join([str(x) for x in input_mask]))
            logger.info("segment_ids: %s", " ".join([str(x) for x in segment_ids]))
            logger.info("label_ids: %s", " ".join([str(x) for x in label_ids]))

        features.append(
            InputFeatures(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids, label_ids=label_ids)
        )
    return features


def get_labels(path):
    if path:
        with open(path, "r") as f:
            labels = f.read().splitlines()
        if "O" not in labels:
            labels = ["O"] + labels
        return labels
    else:
        return ["O", "B-MISC", "I-MISC", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]


In [5]:
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Fine-tuning the library models for named entity recognition on CoNLL-2003 (Bert or Roberta). """




#logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_FOR_TOKEN_CLASSIFICATION_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

ALL_MODELS = sum((tuple(conf.pretrained_config_archive_map.keys()) for conf in MODEL_CONFIG_CLASSES), ())

TOKENIZER_ARGS = ["do_lower_case", "strip_accents", "keep_accents", "use_fast"]


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

def evaluate(args, model, tokenizer, labels, pad_token_label_id, mode, prefix=""):
    eval_dataset = load_and_cache_examples(args, tokenizer, labels, pad_token_label_id, mode=mode)

    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly
    eval_sampler = SequentialSampler(eval_dataset) if args.local_rank == -1 else DistributedSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
#     logger.info("***** Running evaluation %s *****", prefix)
#     logger.info("  Num examples = %d", len(eval_dataset))
#     logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    model.eval()
    for batch in eval_dataloader:
        batch = tuple(t.to(args.device) for t in batch)

        with torch.no_grad():
            inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[3]}
            if args.model_type != "distilbert":
                inputs["token_type_ids"] = (
                    batch[2] if args.model_type in ["bert", "xlnet"] else None
                )  # XLM and RoBERTa don"t use segment_ids
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            if args.n_gpu > 1:
                tmp_eval_loss = tmp_eval_loss.mean()  # mean() to average on multi-gpu parallel evaluating

            eval_loss += tmp_eval_loss.item()
        nb_eval_steps += 1
        if preds is None:
            preds = logits.detach().cpu().numpy()
            out_label_ids = inputs["labels"].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)

    eval_loss = eval_loss / nb_eval_steps
    preds = np.argmax(preds, axis=2)

    label_map = {i: label for i, label in enumerate(labels)}

    out_label_list = [[] for _ in range(out_label_ids.shape[0])]
    preds_list = [[] for _ in range(out_label_ids.shape[0])]

    for i in range(out_label_ids.shape[0]):
        for j in range(out_label_ids.shape[1]):
            if out_label_ids[i, j] != pad_token_label_id:
                out_label_list[i].append(label_map[out_label_ids[i][j]])
                preds_list[i].append(label_map[preds[i][j]])

    results = {
        "loss": eval_loss,
        "precision": precision_score(out_label_list, preds_list),
        "recall": recall_score(out_label_list, preds_list),
        "f1": f1_score(out_label_list, preds_list),
    }

    return results, preds_list


def load_and_cache_examples(args, tokenizer, labels, pad_token_label_id, mode):
    if args.local_rank not in [-1, 0] and not evaluate:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

    # Load data features from cache or dataset file
    cached_features_file = os.path.join(
        args.data_dir,
        "cached_{}_{}_{}".format(
            mode, list(filter(None, args.model_name_or_path.split("/"))).pop(), str(args.max_seq_length)
        ),
    )
    if os.path.exists(cached_features_file) and not args.overwrite_cache:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
    else:
        logger.info("Creating features from dataset file at %s", args.data_dir)
        examples = read_examples_from_file(args.data_dir, mode)
        features = convert_examples_to_features(
            examples,
            labels,
            args.max_seq_length,
            tokenizer,
            cls_token_at_end=bool(args.model_type in ["xlnet"]),
            # xlnet has a cls token at the end
            cls_token=tokenizer.cls_token,
            cls_token_segment_id=2 if args.model_type in ["xlnet"] else 0,
            sep_token=tokenizer.sep_token,
            sep_token_extra=bool(args.model_type in ["roberta"]),
            # roberta uses an extra separator b/w pairs of sentences, cf. github.com/pytorch/fairseq/commit/1684e166e3da03f5b600dbb7855cb98ddfcd0805
            pad_on_left=bool(args.model_type in ["xlnet"]),
            # pad on the left for xlnet
            pad_token=tokenizer.pad_token_id,
            pad_token_segment_id=tokenizer.pad_token_type_id,
            pad_token_label_id=pad_token_label_id,
        )
#         if args.local_rank in [-1, 0]:
#             logger.info("Saving features into cached file %s", cached_features_file)
#             torch.save(features, cached_features_file)

    if args.local_rank == 0 and not evaluate:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_ids for f in features], dtype=torch.long)

    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    return dataset


def main(files=None, data_dir=None, model_type=None, model_name_or_path=None, output_dir=None,
         max_seq_length=128, seed=42, do_predict=False, labels="", config_name="", 
         tokenizer_name="", cache_dir="", do_train=False, do_eval=False, 
         evaluate_during_training=False, do_lower_case=False, keep_accents=False,
         strip_accents=False, use_fast=False, per_gpu_train_batch_size=8, per_gpu_eval_batch_size=8,
         gradient_accumulation_steps=1, learning_rate=5e-5, weight_decay=0.0, adam_epsilon=1e-8,
         max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_steps=500,
         save_steps=500, eval_all_checkpoints=False, no_cuda=False, overwrite_output_dir=False,
         overwrite_cache=False, fp16=False, fp16_opt_level="O1", local_rank=-1):
    
#     parser = argparse.ArgumentParser()
    args = argparse.Namespace(data_dir=data_dir, model_type=model_type, model_name_or_path=model_name_or_path,
                               labels=labels, config_name=config_name, tokenizer_name=tokenizer_name,
                               cache_dir=cache_dir, max_seq_length=max_seq_length, do_train=do_train,
                               do_eval=do_eval, do_predict=do_predict, evaluate_during_training=evaluate_during_training,
                               do_lower_case=do_lower_case, keep_accents=keep_accents, strip_accents=strip_accents,
                               use_fast=use_fast, per_gpu_train_batch_size=per_gpu_train_batch_size,
                               per_gpu_eval_batch_size=per_gpu_eval_batch_size, 
                               gradient_accumulation_steps=gradient_accumulation_steps, learning_rate=learning_rate,
                               weight_decay=weight_decay, adam_epsilon=adam_epsilon, max_grad_norm=max_grad_norm, 
                               num_train_epochs=num_train_epochs, max_steps=max_steps, warmup_steps=warmup_steps,
                               logging_steps=logging_steps, ave_steps=save_steps, eval_all_checkpoints=eval_all_checkpoints,
                               no_cuda=no_cuda, overwrite_output_dir=overwrite_output_dir, overwrite_cache=overwrite_cache,
                               seed=seed, fp16=fp16, fp16_opt_level=fp16_opt_level, local_rank=local_rank,
                               output_dir=output_dir, files=files)

#     if (
#         os.path.exists(args.output_dir)
#         and os.listdir(args.output_dir)
#         and args.do_train
#         and not args.overwrite_output_dir
#     ):
#         raise ValueError(
#             "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
#                 args.output_dir
#             )
#         )

    # Setup CUDA, GPU & distributed training
    if local_rank == -1 or no_cuda:
        device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
        args.n_gpu = 0 if args.no_cuda else torch.cuda.device_count()
    else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
        torch.cuda.set_device(local_rank)
        device = torch.device("cuda", local_rank)
        torch.distributed.init_process_group(backend="nccl")
        args.n_gpu = 1
    args.device = device

    # Setup logging
#     logging.basicConfig(
#         format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
#         datefmt="%m/%d/%Y %H:%M:%S",
#         level=logging.INFO if local_rank in [-1, 0] else logging.WARN,
#     )
#     logger.warning(
#         "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
#         local_rank,
#         device,
#         n_gpu,
#         bool(args.local_rank != -1),
#         fp16,
#     )

    # Set seed
    set_seed(args)

    # Prepare CONLL-2003 task
    labels = get_labels(labels)
    num_labels = len(labels)
    # Use cross entropy ignore index as padding label id so that only real label ids contribute to the loss later
    pad_token_label_id = CrossEntropyLoss().ignore_index

    # Load pretrained model and tokenizer
    if local_rank not in [-1, 0]:
        torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab

    model_type = model_type.lower()
    config = AutoConfig.from_pretrained(
        config_name if config_name else model_name_or_path,
        num_labels=num_labels,
        id2label={str(i): label for i, label in enumerate(labels)},
        label2id={label: i for i, label in enumerate(labels)},
        cache_dir=cache_dir if cache_dir else None,
    )
    tokenizer_args = {k: v for k, v in vars(args).items() if v is not None and k in TOKENIZER_ARGS}
#     logger.info("Tokenizer arguments: %s", tokenizer_args)
    tokenizer = AutoTokenizer.from_pretrained(
        args.tokenizer_name if args.tokenizer_name else args.model_name_or_path,
        cache_dir=args.cache_dir if args.cache_dir else None,
        **tokenizer_args,
    )
    model = AutoModelForTokenClassification.from_pretrained(
        model_name_or_path,
        from_tf=bool(".ckpt" in model_name_or_path),
        config=config,
        cache_dir=args.cache_dir if args.cache_dir else None,
    )

    if args.local_rank == 0:
        torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab

    model.to(args.device)

#     logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, labels, pad_token_label_id, mode="train")
        global_step, tr_loss = train(args, train_dataset, model, tokenizer, labels, pad_token_label_id)
#         logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()
    if args.do_train and (args.local_rank == -1 or torch.distributed.get_rank() == 0):
        # Create output directory if needed
        if not os.path.exists(args.output_dir) and args.local_rank in [-1, 0]:
            os.makedirs(args.output_dir)

#         logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir, **tokenizer_args)
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("pytorch_transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            model = AutoModelForTokenClassification.from_pretrained(checkpoint)
            model.to(args.device)
            result, _ = evaluate(args, model, tokenizer, labels, pad_token_label_id, mode="dev", prefix=global_step)
            if global_step:
                result = {"{}_{}".format(global_step, k): v for k, v in result.items()}
            results.update(result)
        output_eval_file = os.path.join(args.output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as writer:
            for key in sorted(results.keys()):
                writer.write("{} = {}\n".format(key, str(results[key])))
    
    if args.do_predict and args.local_rank in [-1, 0]:
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir, **tokenizer_args)
        model = AutoModelForTokenClassification.from_pretrained(args.output_dir)
        model.to(args.device)
        def applier(row, args, tokenizer):
            final_predictions = []
            for file in preprocess(row.text, 100, 0, 1, [], tokenizer):
                result, predictions = evaluate(args, model, tokenizer, labels, pad_token_label_id, mode=file)

                # Save results
                #final_predictions.append((file, predictions))

                # Save predictions

                example_id = 0
                for line in file.split('\n'):
                    if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                        final_predictions.append(line)
                        if not predictions[example_id]:
                            example_id += 1
                    elif predictions[example_id]:
                        output_line = line.split()[0] + " " + predictions[example_id].pop(0) + "\n"
                        final_predictions.append(output_line)
#                     else:
#                         logger.warning("Maximum sequence length exceeded: No prediction for '%s'.", line.split()[0])

            return final_predictions
        return files.progress_apply(applier, args=(args, tokenizer), axis=1)
    else:
        return 'spooples'

#if __name__ == "__main__":
#    main()


In [ ]:
bert_ner = main(files=df, data_dir="D:\Documenten\Studie\Master\Scriptie\\notaris_bert", model_type="bert", model_name_or_path='bert-base-dutch-cased',
     output_dir='D:\Documenten\Studie\Master\Scriptie\\notaris_bert\output', max_seq_length=128, save_steps=750, seed=1, do_predict=True)


In [9]:
full_bert = pd.read_csv('full_bert.csv')
notebook.tqdm.pandas()

D:\Anaconda\lib\site-packages\tqdm\std.py:651: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [13]:
def compare(ner, true, distance):
    tp = 0
    fp = 0
    fn = 0
    check = []
    corrector = []
    for x in true:
        # Check Normal Names
        if x in ner:
            tp += 1
            corrector.append(x)
            continue 

        # Check Removed Last Names 
        for y in ner:
            if type(y) == tuple:
                if y[0] == x:
                    tp += 1
                    corrector.append(y[0])
                    break
                elif y[1] == x:
                    tp += 1
                    corrector.append(y[1])
                    break
                else:
                    if fuzz.ratio(x,y[0]) >= distance:
                        tp += 1
                        check.append((x,y[1]))
                        corrector.append(x)
                        break
                    elif fuzz.ratio(x[1],y) >= distance:
                        tp += 1
                        check.append((x[0],y))
                        corrector.append(x)
                        break

            else:
                if fuzz.ratio(x.lower(),y.lower()) >= distance:
                    tp += 1
                    corrector.append(x)
                    check.append((x,y))
                    break
            
        if x not in corrector:
            fn += 1
    fp = len(ner) - tp
    return tp, fp, fn, check

def spacy_name_getter(text, nlp):
    doc = nlp(text)
    holder = []
    for ent in doc.ents:
        if ent.label_ == 'PERSON' and ' ' in ent.text:
#             if re.search("[A-Z][a-z ]+[A-Z]\w+ [A-Z]\w+", ent.text) != None:
#                 holder.append((ent.text, re.sub(' [A-Z]\w+$', '', ent.text)))
#             else:
            holder.append(ent.text)
    return holder

def true_name_getter(true):
    holder = []
    for x in ast.literal_eval(true):
        if x['tussenvoegsel'] != None:
            holder.append(x['voornaam'] + " " + x['tussenvoegsel'] + " " + x['achternaam'])
        elif x['voornaam'] and x['achternaam'] != None:
            holder.append(x['voornaam'] + " " + x['achternaam'])
    return holder

def bertje_test(ner_clean):
    holder = []
    prev = None
    for x in ast.literal_eval(ner_clean):
        if x[-1].strip() == 'B-PER':
            holder.append(x[0])
            prev = x[-1].strip()
        elif x[-1].strip() == 'I-PER':
            if prev == 'B-PER' or prev == 'I-PER':
                holder[-1] += ' ' + x[0]
                prev = x[-1].strip()
            else:
                holder.append(x[0])
                prev = x[-1].strip()
        else:
            prev = x[-1].strip()
        
    return holder

def NER_test(row, model, levenshtein, nlp):
    if model == 'SpaCy':
        ner = spacy_name_getter(row.text, nlp)
    if model == 'BERTje':
        ner = bertje_test(row.ner_clean)
    true = true_name_getter(row.namen)
    tp, fp, fn, check = compare(ner, true, levenshtein)
    #return {'tp' : tp, 'fp': fp, 'fn': fn}
    return {'tp' : tp, 'fp': fp, 'fn': fn, 'check': check}

In [14]:
ner_result = full_bert.progress_apply(NER_test, args=('BERTje', 90, 'hoi'), axis=1)

In [16]:
total_fp = sum([x['fp'] for x in ner_result])
total_tp = sum([x['tp'] for x in ner_result])
total_fn = sum([x['fn'] for x in ner_result])
recall = total_tp / (total_tp + total_fn)
precision = total_tp / (total_tp + total_fp)
print('Recall: ' + str(recall))
print('Precision: ' + str(precision))
print('F1: ' + str(2 * ((precision * recall) / (precision + recall))))

Recall: 0.5380159858175538
Precision: 0.07222029611059938
F1: 0.1273463245587698


In [19]:
recall = 0.4155395595494628
precision = 0.10116569370495634
print('Recall: ' + str(recall))
print('Precision: ' + str(precision))
print('F1: ' + str(2 * ((precision * recall) / (precision + recall))))

Recall: 0.4155395595494628
Precision: 0.10116569370495634
F1: 0.16271693596648715


In [ ]:
total_fp,total_tp,total_fn, total_tp / (total_tp + total_fp), total_tp / (total_tp + total_fn)

### BERTje Tests

#### BERTje SpaCy
TP:   
FP:   
FN:   
Recall: 0.  
Precision: 0.  

#### BERTje met Levenshtein distance 2 en achternaam removal
TP: 48911  
FP:   438764  
FN:  29461  
Recall:  0.6240876843770734  
Precision: 0.1002942533449531

#### BERTje met Levenshtein distance 3 en achternaam removal
TP: 53766  
FP: 434134  
FN: 26860  
Recall: 0.6668568451864163  
Precision: 0.1101988112318098  

In [ ]:
def bertje_loc(ner_clean):
    holder = []
    prev = None
    for x in ast.literal_eval(ner_clean):
        if x[-1].strip() == 'B-LOC':
            holder.append(x[0])
            prev = x[-1].strip()
        elif x[-1].strip() == 'I-LOC':
            if prev == 'B-LOC' or prev == 'I-LOC':
                holder[-1] += ' ' + x[0]
                prev = x[-1].strip()
            else:
                holder.append(x[0])
                prev = x[-1].strip()
        else:
            prev = x[-1].strip()
        
    return holder